<a href="https://colab.research.google.com/github/oneir0mancer/stable-diffusion-diffusers-colab-ui/blob/main/sd_diffusers_colab_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install --upgrade diffusers accelerate transformers xformers safetensors
!mkdir -p outputs/{txt2img,img2img}

In [ ]:
!git clone https://github.com/oneir0mancer/stable-diffusion-diffusers-colab-ui.git StableDiffusionUi
!git clone https://huggingface.co/embed/negative /content/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/Lora/positive

In [ ]:
import torch
from diffusers import DiffusionPipeline

output_index = 0
generator = torch.Generator()

# Creating model pipeline
You can use most from [huggingface](https://huggingface.co). Just make sure they are in diffusers format, check their **Files** and see if there is a `model_index.json`.

I made a simple index wih popular models, so you can just render UI and choose a model from dropdown. Some models require trigger word in the prompt.

For weights in Automatic111 format (**.ckpt** or **.safetensors** but without model_index.json), you'll need to download them and [convert using scripts](#scrollTo=kRUTJXoBc4tG&line=4&uniqifier=1).

In [ ]:
#@title Render model choice UI
from StableDiffusionUi.ColabUI.HugginfaceModelIndex import HugginfaceModelIndex

model_index = HugginfaceModelIndex("/content/StableDiffusionUi/model_index.json")
model_index.render()

In [ ]:
#@title Load chosen model
#@markdown Alternatively you can just paste huggingface model_id or path to model folder here:

path_to_model = ""  #@param {type: "string"}

if path_to_model != "": model_id = path_to_model
else: model_id = model_index.get_model_id()

pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = None

In [ ]:
#@title (Optional) Change sampler 
from diffusers import EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler, UniPCMultistepScheduler

choose_sampler = "Euler A" #@param ["Euler A", "DPM++", "DPM++ Karras", "UniPC"]

if choose_sampler == "Euler A":
    solver = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
elif choose_sampler == "DPM++":
    solver = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
elif choose_sampler == "DPM++ Karras":
    solver = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    solver.use_karras_sigmas = True
elif choose_sampler == "UniPC":
    solver = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

pipe.scheduler = solver
print(f"Sampler '{choose_sampler}' chosen")

## VAE

Models loaded from huggingface should have proper VAE. But you also can load VAE from any other repository from [huggingface](https://huggingface.co), just paste their **model_id** and subfolder (usually just "vae"). 

Or you can load it in Automatic111 format and convert using scripts.

In [ ]:
#@title (Optional) Load VAE

from diffusers import AutoencoderKL

vae_id_or_path = "runwayml/stable-diffusion-v1-5"  #@param {type: "string"}
vae_subfolder = "vae"    #@param {type: "string"}

vae = AutoencoderKL.from_pretrained(vae_id_or_path, subfolder=vae_subfolder, torch_dtype=torch.float16).to("cuda")
pipe.vae = vae

## Textual inversions and LoRAs
This is work in progress. 

You can load any textual inversions using 
```
pipe.load_textual_inversion("path/to/dir", weight_name="filename.pt")
```
To use them, you need to add theit token to a prompt like this `<filename>`.

Note, that it seems that if you change VAE after loading textual inversions, using them will degrade generated images.

In [ ]:
#@title (Optional) Load textual inversions
#@markdown Load every embedding with **.pt** extension from **embeddings** folder.

import os

for path, subdirs, files in os.walk("/content/embeddings/"):
    for name in files:
        try:
            if os.path.splitext(name)[1] != ".pt": continue
            pipe.load_textual_inversion(path, weight_name=name)
            print(path, name)
        except: pass

# Generating images

In [ ]:
#@title Render UI
#@markdown You don't need to run this cell again unless you want to change these settings
save_images = True #@param {type:"boolean"}
display_previewes = True    #@param {type:"boolean"}
#@markdown ---
from  StableDiffusionUi.ColabUI.DiffusionPipelineUI import DiffusionPipelineUI

ui = DiffusionPipelineUI()
ui.render()

In [ ]:
#@title Run this cell to generate images
results = ui.generate(pipe, generator)

if display_previewes:
    ui.display_image_previews(results.images)

if save_images:
    for image in results.images:
        image.save(f"outputs/txt2img/{output_index:05}.png")
        print(f"outputs/txt2img/{output_index:05}.png")
        output_index += 1

#Utils

In [ ]:
#@title Image viewer
#@markdown Run this cell to view last results in full size
from IPython.display import clear_output
import ipywidgets as widgets

slider = widgets.IntSlider(max=len(results.images)-1)

def handler(change):
    slider.max = len(results.images)-1
    if change.new > slider.max: change.new = slider.max
    clear_output(wait=True)
    display(slider, results.images[change.new])

slider.observe(handler, names='value')

display(slider, results.images[slider.value])

# Converting models from ckpt
A lot of models from huggingface/civitai use ckpt/safetensors format, that is not supported by diffusers pipelines.

So we want to download them, and then convert to diffusers format using conversion scripts. That will allow us to load them, using [this cell](#scrollTo=IBdm3HvI02Yo).

If after conversion results look bad, try changing sampler.

In [ ]:
#@title Install dependencies
#@markdown You may need to restart runtime after that: `Runtime -> Restart Runtime`
!wget https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
!apt -y install -qq aria2
!pip install pytorch-lightning
!pip install omegaconf

In [ ]:
#@title (Optional) Download ckpt file
#@markdown Download ckpt/safetensors file from huggingface url. 

#@markdown A good place to find these urls is  [camenduru colabs repo](https://https://github.com/camenduru/stable-diffusion-webui-colab), just look for a line starting with `!aria2c ...`

#@markdown ---
import os

url = "https://huggingface.co/mekabu/MagicalMix_v2/resolve/main/MagicalMix_v2.safetensors" #@param {type:"string"}
ckpt_dump_folder = "/content/models_sd/"    #@param {type:"string"}
model_name = url.split('/')[-1]

bashCommand = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {ckpt_dump_folder} -o {model_name}"
os.system(bashCommand)

In [ ]:
#@title (Optional) Or just upload file
from google.colab import files
f = files.upload()

ckpt_dump_folder = "/content/"
for key in f.keys():
    model_name = key
    break

In [ ]:
#@title Run conversion script
#@markdown Paste a path to where you want this script to cache a model into `dump_path`.

#@markdown Keep `override_path` empty unless you uploaded your file to some custom directory.

import os
import torch
from diffusers.pipelines.stable_diffusion.convert_from_ckpt import download_from_original_stable_diffusion_ckpt

from_safetensors = True #@param {type:"boolean"}
override_path = "" #@param {type:"string"}
if override_path != "":
    checkpoint_path = override_path
else:
    checkpoint_path = os.path.join(ckpt_dump_folder, model_name)

pipe = download_from_original_stable_diffusion_ckpt(
        checkpoint_path=checkpoint_path,
        original_config_file = "/content/v1-inference.yaml",
        from_safetensors=from_safetensors,
    )

dump_path="models/ModelName/" #@param {type:"string"}
pipe.save_pretrained(dump_path, safe_serialization=from_safetensors)

pipe = pipe.to("cuda")
pipe.safety_checker = None
pipe.to(torch_dtype=torch.float16)

#Converting VAE


In [ ]:
#@title (Optional) Download vae file
#@markdown Basically, the same thing as model files

#@markdown ---
import os

url = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt" #@param {type:"string"}
ckpt_dump_folder = "/content/models_sd/"    #@param {type:"string"}
model_name = url.split('/')[-1]

bashCommand = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {ckpt_dump_folder} -o {model_name}"
os.system(bashCommand)

In [ ]:
#@markdown Paste a path to where you want this script to cache a vae into `dump_path`.
from_safetensors = True #@param {type:"boolean"}
dump_path="vae/VaeName/" #@param {type:"string"}

checkpoint_path = os.path.join(ckpt_dump_folder, model_name)
bashCommand = f"python /content/scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path {checkpoint_path} --dump_path={dump_path}"
if from_safetensors:
    bashCommand += " --from_safetensors"
os.system(bashCommand)